In [21]:
import cv2
import numpy as np
from ultralytics import YOLO
from polygon_utils import PolygonUtils


In [24]:

VIDEO_PATH  = r"C:\Users\Alex\Desktop\HF_YOLO_CAR_DETECTOR\cvtest.avi"

In [12]:

# грузим актуальную nano модель YOLO 11 версии
model = YOLO("yolo11n.pt")


## Отрисовываем зону въезда (устраняем ложные срабатывания на транспорт, находящийся на заднем фоне и на парковке) и детектируем транспорт, которой пересекает данную зону. В углу экрана выводим количество ТС, находящихся непосредственно перед шлагбаумами (от 0 до 2). Для закрытия окна нужно нажать 'Q'

In [25]:

cap = cv2.VideoCapture(VIDEO_PATH)

# Ресайз для YOLO
resized_shape = (640, 480)
display_shape = (1080, 720)

# Орнигинальный полигон зоны въезда
original_polygon = np.array([
    [0, 483], [1680, 460], [1680, 1400], [0, 1400]
], dtype=np.int32)

# Получаем размер оригинального кадра
ret, frame = cap.read()
if not ret:
    raise RuntimeError("Не удалось прочитать первый кадр")
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  

orig_h, orig_w = frame.shape[:2]
resized_w, resized_h = 640, 480

scale_x = resized_w / orig_w
scale_y = resized_h / orig_h

# Масштабируем координаты полигона
polygon = np.array([
    [int(x * scale_x), int(y * scale_y)] for x, y in original_polygon
], dtype=np.int32)



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Ресайз под модель 
    frame_resized = cv2.resize(frame, resized_shape)
    output_frame = frame_resized.copy()

    # Отрисовка зоны въезда
    PolygonUtils.draw_dashed_polygon(output_frame, polygon)

    # YOLO
    results = model(frame_resized)[0]

    boxes = []
    scores = []

    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, score, cls = box
        class_name = model.names[int(cls)]

        if class_name in ['car', 'truck', 'motorcycle', 'bus']:
            if PolygonUtils.box_intersects_polygon((x1, y1, x2, y2), polygon):
                boxes.append([x1, y1, x2, y2])
                scores.append(score)

    # применяем NMS чтобы удалить дубликаты
    indices = cv2.dnn.NMSBoxes(
        bboxes=[[int(x1), int(y1), int(x2 - x1), int(y2 - y1)] for x1, y1, x2, y2 in boxes],
        scores=scores,
        score_threshold=0.3,
        nms_threshold=0.4
    )

    indices = indices[:2] if len(indices) > 2 else indices  

    vehicle_count = 0
    for idx in indices:
        idx = idx[0] if isinstance(idx, (tuple, list, np.ndarray)) else idx
        x1, y1, x2, y2 = map(int, boxes[idx])
        vehicle_count += 1
        cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(output_frame, "vehicle", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # вывод статуса
    if vehicle_count == 0:
        status_text = "No vehicles detected in entry zone"
    elif vehicle_count == 1:
        status_text = "1 vehicle in entry zone"
    else:
        status_text = "2 vehicles in entry zone"

    cv2.putText(output_frame, status_text, (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)


    display_frame = cv2.resize(output_frame, display_shape)
    cv2.imshow("Detection", display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 2 persons, 3 cars, 75.3ms
Speed: 85.9ms preprocess, 75.3ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 3 cars, 13.5ms
Speed: 1.3ms preprocess, 13.5ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 5 cars, 10.5ms
Speed: 1.3ms preprocess, 10.5ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 3 cars, 12.0ms
Speed: 1.3ms preprocess, 12.0ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 3 cars, 11.0ms
Speed: 1.3ms preprocess, 11.0ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 3 cars, 13.1ms
Speed: 1.5ms preprocess, 13.1ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 4 cars, 14.6ms
Speed: 1.2ms preprocess, 14.6ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 4 cars, 10.7ms
Speed: 1.7ms pre